In [1]:
import numpy as np
import toolbox as tb

### Generate date

In [2]:
np.random.seed(43)
N = 20

A = np.random.randint(-20, 20, size=(4,4))/10
cov = np.dot(A, A.transpose())
cov

array([[ 6.73, -1.5 ,  4.03,  3.5 ],
       [-1.5 ,  4.82, -5.09, -4.9 ],
       [ 4.03, -5.09,  7.15,  6.86],
       [ 3.5 , -4.9 ,  6.86,  6.63]])

In [3]:
X = np.random.multivariate_normal([1,2,3,4], cov, size=N)
error = np.random.normal(0,1.5,size=N)
Y = 2*X[:,0] - 5*X[:,1] + X[:,2] + 0.3*X[:,3] + error

### Fit multivariate linear model

In [4]:
tb.linear_model(X, Y, add_constant=False)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.996
Model:                            OLS   Adj. R-squared (uncentered):              0.995
Method:                 Least Squares   F-statistic:                              947.8
Date:                Fri, 05 Mar 2021   Prob (F-statistic):                    8.72e-19
Time:                        12:12:26   Log-Likelihood:                         -28.294
No. Observations:                  20   AIC:                                      64.59
Df Residuals:                      16   BIC:                                      68.57
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             3.1023      0.437      7.092      0.000       2.175       4.030
x2            -6.4428      0.591    -10.908      0.000      -7.695      -5.191
x3            -7.1626      2.865     -2.500      0.024     -13.235      -1.090
x4             6.9042      2.339      2.951      0.009       1.945      11.863
==============================================================================
Omnibus:                        1.391   Durbin-Watson:                   2.427
Prob(Omnibus):                  0.499   Jarque-Bera (JB):                1.093
Skew:                          -0.354   Prob(JB):                        0.579
Kurtosis:                       2.099   Cond. No.                         89.3
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Successive orthogonalization approach - deconstructed

In [5]:
(x0, x1, x2, x3) = (X[:,0], X[:,1], X[:,2], X[:,3])

In [6]:
x0.dot(x1), x0.dot(x2), x1.dot(x2)

(53.9564376763056, 121.38349370639646, 34.454796297421126)

In [7]:
from typing import List

def gamma_factor(reference_direction, vec):
    return (np.dot(reference_direction, vec) / np.dot(reference_direction, reference_direction))

def gram_schmidt(vecs: List[np.array]):
    results = []
    for i in range(len(vecs)):
        new_vec = vecs[i].copy()
        for j in range(0,i):
            new_vec -=  gamma_factor(results[j], vecs[i])* results[j]
        results.append(new_vec)
    return results

Will get the x4 coefficient
* first do gram schmidt on x0, x1, x2

In [8]:
z0, z1, z2 = gram_schmidt([x0, x1, x2])

In [9]:
# sanity check
z0.dot(z1), z0.dot(z2), z1.dot(z2)

(2.6645352591003757e-15, 1.7430501486614958e-14, -8.881784197001252e-15)

Now project project x3 onto z1, z2, z3. (This is not quite a projection, because it contains an extra normalization for the projected direction vector)

In [10]:
gamma03 = gamma_factor(z0, x3)
gamma13 = gamma_factor(z1, x3)
gamma23 = gamma_factor(z2, x3)

gamma03, gamma13, gamma23

(1.4306431710219687, 0.04650218844732895, 1.223726129762212)

Another consistency check: since z0, z1, and z2 are orthogonal, regressing x3 on them should result in the same values:

In [11]:
Z = np.hstack([z0.reshape(-1,1), z1.reshape(-1,1), z2.reshape(-1,1)])
tb.linear_model(Z, x3, add_constant=False)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                              9099.
Date:                Fri, 05 Mar 2021   Prob (F-statistic):                    1.93e-27
Time:                        12:12:32   Log-Likelihood:                          16.428
No. Observations:                  20   AIC:                                     -26.86
Df Residuals:                      17   BIC:                                     -23.87
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.4306      0.011    124.709      0.000       1.406       1.455
x2             0.0465      0.009      5.433      0.000       0.028       0.065
x3             1.2237      0.011    108.234      0.000       1.200       1.248
==============================================================================
Omnibus:                        1.016   Durbin-Watson:                   2.723
Prob(Omnibus):                  0.602   Jarque-Bera (JB):                0.765
Skew:                           0.053   Prob(JB):                        0.682
Kurtosis:                       2.048   Cond. No.                         1.34
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Finally to obtain the *multivariate* regression coefficient for Y on x3, we get the residual from the above projections and regress Y on that

In [12]:
z3 = x3 - gamma03*z0 - gamma13*z1 - gamma23*z2

In [13]:
tb.linear_model(z3, Y, add_constant=False)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.002
Model:                            OLS   Adj. R-squared (uncentered):             -0.050
Method:                 Least Squares   F-statistic:                            0.04357
Date:                Fri, 05 Mar 2021   Prob (F-statistic):                       0.837
Time:                        12:12:33   Log-Likelihood:                         -82.993
No. Observations:                  20   AIC:                                      168.0
Df Residuals:                      19   BIC:                                      169.0
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             6.9042     33.076      0.209      0.837     -62.324      76.133
==============================================================================
Omnibus:                        0.242   Durbin-Watson:                   1.913
Prob(Omnibus):                  0.886   Jarque-Bera (JB):                0.332
Skew:                           0.220   Prob(JB):                        0.847
Kurtosis:                       2.547   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

which equivalently is just the projection of Y on z3:

In [14]:
gamma_factor(z3, Y)

6.90417306413922

This can indeed be done for each input variable

## Successive orthogonalization appraoch - all together

In [15]:
def get_residual_for(vec, other_vecs: List[np.array]):
    zs = gram_schmidt(other_vecs)
    res = vec.copy()
    for z in zs:
        res -= gamma_factor(z,vec)*z
    return res

def get_multivariate_regression_coeff(Y, vec, other_vecs):
    resid = get_residual_for(vec, other_vecs)
    return gamma_factor(resid, Y)

In [16]:
inputs = [x0, x1, x2, x3]

In [17]:
for vec in inputs:
    print(get_multivariate_regression_coeff(Y, vec, [v for v in inputs if not np.isclose(v, vec).all()]))

3.1023263953811218
-6.442841030182411
-7.1625682861932365
6.90417306413922


For comparison

In [18]:
tb.linear_model(X, Y, add_constant=False)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.996
Model:                            OLS   Adj. R-squared (uncentered):              0.995
Method:                 Least Squares   F-statistic:                              947.8
Date:                Fri, 05 Mar 2021   Prob (F-statistic):                    8.72e-19
Time:                        12:12:37   Log-Likelihood:                         -28.294
No. Observations:                  20   AIC:                                      64.59
Df Residuals:                      16   BIC:                                      68.57
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             3.1023      0.437      7.092      0.000       2.175       4.030
x2            -6.4428      0.591    -10.908      0.000      -7.695      -5.191
x3            -7.1626      2.865     -2.500      0.024     -13.235      -1.090
x4             6.9042      2.339      2.951      0.009       1.945      11.863
==============================================================================
Omnibus:                        1.391   Durbin-Watson:                   2.427
Prob(Omnibus):                  0.499   Jarque-Bera (JB):                1.093
Skew:                          -0.354   Prob(JB):                        0.579
Kurtosis:                       2.099   Cond. No.                         89.3
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Single pass -  QR decomposition

Do gramm schmidt

In [19]:
xs = [x0, x1, x2, x3]
zs = gram_schmidt(xs)

In [52]:
Z = np.stack(zs).T

In [53]:
def gamma(i,j):
    if i>j:
        return 0
    else:
        return gamma_factor(zs[i], xs[j])

In [54]:
Gamma = np.zeros((4,4))
for i in range(4):
    for j in range(4):
        Gamma[i,j] = gamma(i,j)

In [55]:
Gamma

array([[ 1.        ,  0.53291728,  1.19888126,  1.43064317],
       [ 0.        ,  1.        , -0.16621595,  0.04650219],
       [ 0.        ,  0.        ,  1.        ,  1.22372613],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

Check $ X = Z\Gamma $

In [56]:
np.allclose(Z@Gamma,X)

True

Turn into QR

In [78]:
# D = np.round(Z.T.dot(Z),10)
D = np.diag([np.sqrt(z.dot(z)) for z in zs])
D_inv = np.diag([1/np.sqrt(z.dot(z)) for z in zs])

In [79]:
D_inv@D

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [80]:
Q = Z@D_inv
R = D@Gamma

check $X = QR$

In [81]:
np.allclose(Q@R,X)

True

In [85]:
Q.shape, R.shape

((20, 4), (4, 4))

check $Q^TQ = I$

In [86]:
np.allclose(Q.T@Q, np.eye(4))

True

Now solve: $ R\hat{\beta} = Q^Ty $

In [88]:
b4 = (Q.T@Y)[-1]/R[-1,-1]
b4

6.904173064142121

In [92]:
b3 =  ((Q.T@Y)[-2] - b4*R[-2,-1])/R[-2,-2]
b3

-7.162568286196149

etc. etc

# Cleaner QR approach

Note: numpy has a QR decomposition

In [122]:
Q, R = np.linalg.qr(X)

In [123]:
out = Q.T@Y

In [124]:
betas = np.zeros(4)

In [125]:
for i in [3,2,1,0]:
    pivot = R[i,i]
    resp = out[i].copy()
    for j in range(i,4):
        resp-=betas[j]*R[i,j]
    betas[i] = (resp)/pivot

In [126]:
betas

array([ 3.1023264 , -6.44284103, -7.16256829,  6.90417306])

For comparison

In [127]:
tb.linear_model(X, Y, add_constant=False)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.996
Model:                            OLS   Adj. R-squared (uncentered):              0.995
Method:                 Least Squares   F-statistic:                              947.8
Date:                Fri, 05 Mar 2021   Prob (F-statistic):                    8.72e-19
Time:                        13:54:24   Log-Likelihood:                         -28.294
No. Observations:                  20   AIC:                                      64.59
Df Residuals:                      16   BIC:                                      68.57
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             3.1023      0.437      7.092      0.000       2.175       4.030
x2            -6.4428      0.591    -10.908      0.000      -7.695      -5.191
x3            -7.1626      2.865     -2.500      0.024     -13.235      -1.090
x4             6.9042      2.339      2.951      0.009       1.945      11.863
==============================================================================
Omnibus:                        1.391   Durbin-Watson:                   2.427
Prob(Omnibus):                  0.499   Jarque-Bera (JB):                1.093
Skew:                          -0.354   Prob(JB):                        0.579
Kurtosis:                       2.099   Cond. No.                         89.3
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## timing

In [132]:
%%timeit
Q, R = np.linalg.qr(X)
out = Q.T@Y
betas = np.zeros(4)
for i in [3,2,1,0]:
    pivot = R[i,i]
    resp = out[i].copy()
    for j in range(i,4):
        resp-=betas[j]*R[i,j]
    betas[i] = (resp)/pivot

52.8 µs ± 2.29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [133]:
%%timeit
tb.linear_model(X, Y, add_constant=False, verbose=False)

298 µs ± 6.06 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Certainly the one below is computing more stuff